<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-Scripts-for-Cluster-Data" data-toc-modified-id="Analysis-Scripts-for-Cluster-Data-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Analysis Scripts for Cluster Data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation" data-toc-modified-id="Dictionary-Creation-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Dictionary Creation</a></span></li></ul></li></ul></li><li><span><a href="#Converged-Experiments-with-0%-sensor-noise" data-toc-modified-id="Converged-Experiments-with-0%-sensor-noise-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Converged Experiments with 0% sensor noise</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-successful-seeds-with-0-%-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-successful-seeds-with-0-%-sensor-noise-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Dictionary Creation for successful seeds with 0 % sensor noise</a></span></li><li><span><a href="#Dictionaries-for-timestep-and-density-ratios" data-toc-modified-id="Dictionaries-for-timestep-and-density-ratios-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Dictionaries for timestep and density ratios</a></span></li></ul></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Working-robot-ratio" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Working-robot-ratio-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Boxplot Time to converge vs. Working robot ratio</a></span></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Density" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Density-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Boxplot Time to converge vs. Density</a></span></li></ul></li><li><span><a href="#Converged-Experiments-with-10%-sensor-noise" data-toc-modified-id="Converged-Experiments-with-10%-sensor-noise-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Converged Experiments with 10% sensor noise</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-successful-seeds-with-10-%-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-successful-seeds-with-10-%-sensor-noise-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Dictionary Creation for successful seeds with 10 % sensor noise</a></span></li><li><span><a href="#Dictionaries-for-timestep-and-density-ratios" data-toc-modified-id="Dictionaries-for-timestep-and-density-ratios-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Dictionaries for timestep and density ratios</a></span></li></ul></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Working-robot-ratio" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Working-robot-ratio-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Boxplot Time to converge vs. Working robot ratio</a></span></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Density" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Density-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Boxplot Time to converge vs. Density</a></span></li></ul></li><li><span><a href="#Converged-Experiments-with-20%-sensor-noise" data-toc-modified-id="Converged-Experiments-with-20%-sensor-noise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Converged Experiments with 20% sensor noise</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-successful-seeds-with-20-%-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-successful-seeds-with-20-%-sensor-noise-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Dictionary Creation for successful seeds with 20 % sensor noise</a></span></li><li><span><a href="#Dictionaries-for-timestep-and-density-ratios" data-toc-modified-id="Dictionaries-for-timestep-and-density-ratios-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Dictionaries for timestep and density ratios</a></span></li></ul></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Working-robot-ratio" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Working-robot-ratio-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Boxplot Time to converge vs. Working robot ratio</a></span></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Density" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Density-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Boxplot Time to converge vs. Density</a></span></li></ul></li><li><span><a href="#Converged-Experiments-with-40%-sensor-noise" data-toc-modified-id="Converged-Experiments-with-40%-sensor-noise-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Converged Experiments with 40% sensor noise</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-successful-seeds-with-40-%-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-successful-seeds-with-40-%-sensor-noise-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Dictionary Creation for successful seeds with 40 % sensor noise</a></span></li><li><span><a href="#Dictionaries-for-timestep-and-density-ratios" data-toc-modified-id="Dictionaries-for-timestep-and-density-ratios-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>Dictionaries for timestep and density ratios</a></span></li></ul></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Working-robot-ratio" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Working-robot-ratio-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Boxplot Time to converge vs. Working robot ratio</a></span></li><li><span><a href="#Boxplot-Time-to-converge-vs.-Density" data-toc-modified-id="Boxplot-Time-to-converge-vs.-Density-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Boxplot Time to converge vs. Density</a></span></li></ul></li><li><span><a href="#Heatmap-Successful-Experiments" data-toc-modified-id="Heatmap-Successful-Experiments-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Heatmap Successful Experiments</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-all-successful-seeds-separated-by-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-all-successful-seeds-separated-by-sensor-noise-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>Dictionary Creation for all successful seeds separated by sensor noise</a></span><ul class="toc-item"><li><span><a href="#Pandas-Count-of-successful-experiments" data-toc-modified-id="Pandas-Count-of-successful-experiments-5.0.1.1"><span class="toc-item-num">5.0.1.1&nbsp;&nbsp;</span>Pandas Count of successful experiments</a></span></li><li><span><a href="#Pandas-Mean-timestep-for-successful-experiments" data-toc-modified-id="Pandas-Mean-timestep-for-successful-experiments-5.0.1.2"><span class="toc-item-num">5.0.1.2&nbsp;&nbsp;</span>Pandas Mean timestep for successful experiments</a></span></li></ul></li></ul></li><li><span><a href="#Heatmap-Number-of-Successful-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]" data-toc-modified-id="Heatmap-Number-of-Successful-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Heatmap Number of Successful Experiments [Sensor Noise vs. Defecting Ratio]</a></span></li><li><span><a href="#Heatmap-Timestep-of-Successful-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]" data-toc-modified-id="Heatmap-Timestep-of-Successful-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Heatmap Timestep of Successful Experiments [Sensor Noise vs. Defecting Ratio]</a></span></li></ul></li><li><span><a href="#Heatmap-Failed-Experiments" data-toc-modified-id="Heatmap-Failed-Experiments-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Heatmap Failed Experiments</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Dictionary-Creation-for-all-failed-seeds-separated-by-sensor-noise" data-toc-modified-id="Dictionary-Creation-for-all-failed-seeds-separated-by-sensor-noise-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Dictionary Creation for all failed seeds separated by sensor noise</a></span><ul class="toc-item"><li><span><a href="#Pandas-Count-of-failed-experiments" data-toc-modified-id="Pandas-Count-of-failed-experiments-6.0.1.1"><span class="toc-item-num">6.0.1.1&nbsp;&nbsp;</span>Pandas Count of failed experiments</a></span></li><li><span><a href="#Pandas-Mean-timestep-for-failed-experiments" data-toc-modified-id="Pandas-Mean-timestep-for-failed-experiments-6.0.1.2"><span class="toc-item-num">6.0.1.2&nbsp;&nbsp;</span>Pandas Mean timestep for failed experiments</a></span></li></ul></li><li><span><a href="#Dictionary-Creation-for-counts-of-failed-robots" data-toc-modified-id="Dictionary-Creation-for-counts-of-failed-robots-6.0.2"><span class="toc-item-num">6.0.2&nbsp;&nbsp;</span>Dictionary Creation for counts of failed robots</a></span><ul class="toc-item"><li><span><a href="#Pandas-Count-of-failed-robots-vs.-ratio-of-robots" data-toc-modified-id="Pandas-Count-of-failed-robots-vs.-ratio-of-robots-6.0.2.1"><span class="toc-item-num">6.0.2.1&nbsp;&nbsp;</span>Pandas Count of failed robots vs. ratio of robots</a></span></li><li><span><a href="#Pandas-Count-of-failed-robots-vs.-number-of-working-robots" data-toc-modified-id="Pandas-Count-of-failed-robots-vs.-number-of-working-robots-6.0.2.2"><span class="toc-item-num">6.0.2.2&nbsp;&nbsp;</span>Pandas Count of failed robots vs. number of working robots</a></span></li></ul></li></ul></li><li><span><a href="#Heatmap-Number-of-Failed-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]" data-toc-modified-id="Heatmap-Number-of-Failed-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Heatmap Number of Failed Experiments [Sensor Noise vs. Defecting Ratio]</a></span></li><li><span><a href="#Heatmap-Timestep-of-Failed-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]" data-toc-modified-id="Heatmap-Timestep-of-Failed-Experiments-[Sensor-Noise-vs.-Defecting-Ratio]-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Heatmap Timestep of Failed Experiments [Sensor Noise vs. Defecting Ratio]</a></span></li><li><span><a href="#Heatmap-Number-of-Failed-Robots-[Sensor-Noise-vs.-Defecting-Ratio]" data-toc-modified-id="Heatmap-Number-of-Failed-Robots-[Sensor-Noise-vs.-Defecting-Ratio]-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Heatmap Number of Failed Robots [Sensor Noise vs. Defecting Ratio]</a></span></li><li><span><a href="#Heatmap-Number-of-Failed-Robots-[Sensor-Noise-vs.-Working-Robots]" data-toc-modified-id="Heatmap-Number-of-Failed-Robots-[Sensor-Noise-vs.-Working-Robots]-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Heatmap Number of Failed Robots [Sensor Noise vs. Working Robots]</a></span></li></ul></li><li><span><a href="#Scatter-Plot-of-Messages-Listened/Ignored" data-toc-modified-id="Scatter-Plot-of-Messages-Listened/Ignored-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Scatter Plot of Messages Listened/Ignored</a></span><ul class="toc-item"><li><span><a href="#Successful-experiments" data-toc-modified-id="Successful-experiments-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Successful experiments</a></span><ul class="toc-item"><li><span><a href="#Dictionaries-for-message-counts-and-final-beliefs" data-toc-modified-id="Dictionaries-for-message-counts-and-final-beliefs-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Dictionaries for message counts and final beliefs</a></span></li><li><span><a href="#Scatter-plot-messages-vs.-num.-robots" data-toc-modified-id="Scatter-plot-messages-vs.-num.-robots-7.1.2"><span class="toc-item-num">7.1.2&nbsp;&nbsp;</span>Scatter plot messages vs. num. robots</a></span></li><li><span><a href="#Scatter-plot-messages-vs.-working-robot-ratio" data-toc-modified-id="Scatter-plot-messages-vs.-working-robot-ratio-7.1.3"><span class="toc-item-num">7.1.3&nbsp;&nbsp;</span>Scatter plot messages vs. working robot ratio</a></span></li><li><span><a href="#Scatter-plot-beliefs-vs.-num.-robots" data-toc-modified-id="Scatter-plot-beliefs-vs.-num.-robots-7.1.4"><span class="toc-item-num">7.1.4&nbsp;&nbsp;</span>Scatter plot beliefs vs. num. robots</a></span></li><li><span><a href="#Scatter-plot-beliefs-vs.-working-robot-ratio" data-toc-modified-id="Scatter-plot-beliefs-vs.-working-robot-ratio-7.1.5"><span class="toc-item-num">7.1.5&nbsp;&nbsp;</span>Scatter plot beliefs vs. working robot ratio</a></span></li></ul></li><li><span><a href="#Failed-experiments" data-toc-modified-id="Failed-experiments-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Failed experiments</a></span><ul class="toc-item"><li><span><a href="#Dictionaries-for-message-counts-and-final-beliefs" data-toc-modified-id="Dictionaries-for-message-counts-and-final-beliefs-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Dictionaries for message counts and final beliefs</a></span></li><li><span><a href="#Scatter-plot-messages-vs.-num.-robots" data-toc-modified-id="Scatter-plot-messages-vs.-num.-robots-7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Scatter plot messages vs. num. robots</a></span></li><li><span><a href="#Scatter-plot-messages-vs.-working-robot-ratio" data-toc-modified-id="Scatter-plot-messages-vs.-working-robot-ratio-7.2.3"><span class="toc-item-num">7.2.3&nbsp;&nbsp;</span>Scatter plot messages vs. working robot ratio</a></span></li><li><span><a href="#Scatter-plot-beliefs-vs.-num.-robots" data-toc-modified-id="Scatter-plot-beliefs-vs.-num.-robots-7.2.4"><span class="toc-item-num">7.2.4&nbsp;&nbsp;</span>Scatter plot beliefs vs. num. robots</a></span></li><li><span><a href="#Scatter-plot-beliefs-vs.-working-robot-ratio" data-toc-modified-id="Scatter-plot-beliefs-vs.-working-robot-ratio-7.2.5"><span class="toc-item-num">7.2.5&nbsp;&nbsp;</span>Scatter plot beliefs vs. working robot ratio</a></span></li></ul></li></ul></li><li><span><a href="#BREAK" data-toc-modified-id="BREAK-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>BREAK</a></span></li></ul></div>

---
# Analysis Scripts for Cluster Data

In [ ]:
import csv
import sys
import os
import re
import numpy as np
import datetime
import pickle
import collections
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import cm 
import seaborn as sns
import pandas as pd
from scipy.interpolate import griddata

% matplotlib inline

---
### Dictionary Creation

In [ ]:
# Nifty trick
class Customdictionary(dict):
    def __setitem__(self,key,value):
        try:
            self[key]
        except KeyError:
            super(Customdictionary,self).__setitem__(key,[])
        self[key].append(value)

In [ ]:
def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True

In [ ]:
def remove_chars(string):
    chars="[^0123456789\.]"
    return re.sub(chars,"",string)

In [ ]:
#Time [0],#Robot [1],#X [2],#Y [3]
#PickedPatternNum [4],#PickedPattern [5] --> PATTERNPICKED
#PickedPatternProb [6] -->PICKEDPROBABILITY
#CorrectPatternProb [7] -->CORRECTPATTERNPROBABILITY
#TotalReceivedMessages [8] -->NUMBEROFRECEIVEDMESSAGES
#TotalPrunedMessages [9] -->NUMBEROFLISTENEDMESSAGES
#TotalLyingMessages [10] -->NUMBEROFLYINGMESSAGESOBTAINED
#TotalLyingMessagesBelieved [11] -->NUMBEROFLYINGMESSAGESLISTENED
#CurrentThreshold [12] -->ROBOTTHRESHOLD

In [ ]:
pickle_dictionary=collections.defaultdict(dict)

pickle_location='../cluster_data/motion_threshold_1212/'

pickle_files=os.listdir(pickle_location)

for i,pkl_file in enumerate(pickle_files):
#         print(pkl_file,"\n")
        fname=pkl_file.split("_")
        
        num_robots=remove_chars(fname[0])
        num_liars=remove_chars(fname[1])
        noise=remove_chars(fname[2])
        density=remove_chars(fname[3])
        seed=remove_chars(fname[4].split(".")[0])
        
        key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
        key_1="seed{}".format(seed)

        data_path=os.path.join(pickle_location,pkl_file)
        
        if os.path.getsize(data_path)>0:      
            with open(data_path,"rb") as f:
                unpickler=pickle.Unpickler(f)
                data=unpickler.load()
                for k,v in data.items():
                    pickle_dictionary[key_0][key_1]=v
        else:
            print("ISSUE: ", pkl_file)

In [ ]:
successful_seeds=[]
failed_seeds=[]

countfailed_robots_failedexp=Customdictionary()

for k1,v1 in pickle_dictionary.items():
    fname=k1.split("_")
    num_robots=int(remove_chars(fname[0]))
    num_liars=int(remove_chars(fname[1]))
    noise=float(remove_chars(fname[2]))
    density=float(remove_chars(fname[3]))
    num_working=num_robots-num_liars
    
    for k2,v2 in v1.items():
        data_length=len(v2)
        if (num_working==data_length):
            successful_seeds.append("_".join((k1,k2)))
        elif (num_working>data_length):
            failed_seeds.append("_".join((k1,k2)))
            failed_robots=num_working-data_length
            countfailed_robots_failedexp["_".join((k1,k2))]=failed_robots
        else:
            print("ERR")
successful_seeds=np.array(successful_seeds)
failed_seeds=np.array(failed_seeds)


**---**

# Converged Experiments with 0% sensor noise

### Dictionary Creation for successful seeds with 0 % sensor noise

In [ ]:
# Creating the needed dictionary
converged_00=Customdictionary()

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    if (noise==0.0):
        converged_00[successful_seeds[ind]]=pickle_dictionary[key_0][key_1]

### Dictionaries for timestep and density ratios

In [ ]:
time_converged_00=collections.defaultdict(dict)
density_converged_00=collections.defaultdict(dict)

for k,v in converged_00.items():
    k_id=k.split("_")
    num_robots=int(remove_chars(k_id[0]))
    num_liars=int(remove_chars(k_id[1]))
    num_working=num_robots-num_liars
    noise=float(remove_chars(k_id[2]))
    density=int(remove_chars(k_id[3]))
    seed=int(remove_chars(k_id[4]))
    
    key="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key2="_".join((key,"seed{}".format(seed)))
    raw_data=np.array(v).reshape((num_working,9))
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
    
    
    x_val=round(float(num_working/num_robots),2) # Working robots
    y_val=timestep
    #1
    if not keys_exists(time_converged_00,x_val,key):
        data=np.array(y_val)
        time_converged_00[x_val][key]=data
    else:
        data=time_converged_00[x_val][key]
        data=np.hstack((data,y_val))
        time_converged_00[x_val][key]=data
    #2
    if not keys_exists(density_converged_00,density,key):
        data=np.array(y_val)
        density_converged_00[density][key]=data
    else:
        data=density_converged_00[density][key]
        data=np.hstack((data,y_val))
        density_converged_00[density][key]=data


---

## Boxplot Time to converge vs. Working robot ratio 
Converged Experiments (0% sensor noise)
Color: Density or Number of Robots

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(0.4,1)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in time_converged_00.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('Density of {}'.format(density))
        boxplot_x=key+(0.01*counter)-0.015
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[density]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.005)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[0.2,0.25,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.88,0.9,0.95,1.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Working Robot Ratio')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 

---

## Boxplot Time to converge vs. Density
Converged Experiments (0% sensor noise)
Color: Number of Robots or Number of liars

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(1,10)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in density_converged_00.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('{} liars'.format(num_liars))
        boxplot_x=key+(0.5*counter)-0.5
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[num_liars]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.5)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[1.0,2.0,5.0,10.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Density')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 


**---**

# Converged Experiments with 10% sensor noise

### Dictionary Creation for successful seeds with 10 % sensor noise

In [ ]:
# Creating the needed dictionary
converged_10=Customdictionary()

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    if (noise==0.10):
        converged_10[successful_seeds[ind]]=pickle_dictionary[key_0][key_1]

### Dictionaries for timestep and density ratios

In [ ]:
time_converged_10=collections.defaultdict(dict)
density_converged_10=collections.defaultdict(dict)

for k,v in converged_10.items():
    k_id=k.split("_")
    num_robots=int(remove_chars(k_id[0]))
    num_liars=int(remove_chars(k_id[1]))
    num_working=num_robots-num_liars
    noise=float(remove_chars(k_id[2]))
    density=int(remove_chars(k_id[3]))
    seed=int(remove_chars(k_id[4]))
    
    key="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key2="_".join((key,"seed{}".format(seed)))
    raw_data=np.array(v).reshape((num_working,9))
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
    
    
    x_val=round(float(num_working/num_robots),2) # Working robots
    y_val=timestep
    #1
    if not keys_exists(time_converged_10,x_val,key):
        data=np.array(y_val)
        time_converged_10[x_val][key]=data
    else:
        data=time_converged_10[x_val][key]
        data=np.hstack((data,y_val))
        time_converged_10[x_val][key]=data
    #2
    if not keys_exists(density_converged_10,density,key):
        data=np.array(y_val)
        density_converged_10[density][key]=data
    else:
        data=density_converged_10[density][key]
        data=np.hstack((data,y_val))
        density_converged_10[density][key]=data


---

## Boxplot Time to converge vs. Working robot ratio 
Converged Experiments (10% sensor noise)
Color: Density or Number of Robots

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(0.4,1)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in time_converged_10.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('Density of {}'.format(density))
        boxplot_x=key+(0.01*counter)-0.015
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[density]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.005)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[0.2,0.25,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.88,0.9,0.95,1.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Working Robot Ratio')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 

---

## Boxplot Time to converge vs. Density
Converged Experiments (10% sensor noise)
Color: Number of Robots or Number of liars

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(1,10)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in density_converged_10.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('{} liars'.format(num_liars))
        boxplot_x=key+(0.5*counter)-0.5
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[num_liars]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.5)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[1.0,2.0,5.0,10.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Density')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 


**---**

# Converged Experiments with 20% sensor noise

### Dictionary Creation for successful seeds with 20 % sensor noise

In [ ]:
# Creating the needed dictionary
converged_20=Customdictionary()

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    if (noise==0.20):
        converged_20[successful_seeds[ind]]=pickle_dictionary[key_0][key_1]

### Dictionaries for timestep and density ratios

In [ ]:
time_converged_20=collections.defaultdict(dict)
density_converged_20=collections.defaultdict(dict)

for k,v in converged_20.items():
    k_id=k.split("_")
    num_robots=int(remove_chars(k_id[0]))
    num_liars=int(remove_chars(k_id[1]))
    num_working=num_robots-num_liars
    noise=float(remove_chars(k_id[2]))
    density=int(remove_chars(k_id[3]))
    seed=int(remove_chars(k_id[4]))
    
    key="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key2="_".join((key,"seed{}".format(seed)))
    raw_data=np.array(v).reshape((num_working,9))
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
    
    
    x_val=round(float(num_working/num_robots),2) # Working robots
    y_val=timestep
    #1
    if not keys_exists(time_converged_20,x_val,key):
        data=np.array(y_val)
        time_converged_20[x_val][key]=data
    else:
        data=time_converged_20[x_val][key]
        data=np.hstack((data,y_val))
        time_converged_20[x_val][key]=data
    #2
    if not keys_exists(density_converged_20,density,key):
        data=np.array(y_val)
        density_converged_20[density][key]=data
    else:
        data=density_converged_20[density][key]
        data=np.hstack((data,y_val))
        density_converged_20[density][key]=data


---

## Boxplot Time to converge vs. Working robot ratio 
Converged Experiments (20% sensor noise)
Color: Density or Number of Robots

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(0.4,1)

color_count=0
color_map=cm.Set1(np.linspace(0,1,51))

for key,value in time_converged_20.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('Density of {}'.format(density))
        boxplot_x=key+(0.01*counter)-0.015
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[density]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.005)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[0.2,0.25,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.88,0.9,0.95,1.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Working Robot Ratio')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 

---

## Boxplot Time to converge vs. Density
Converged Experiments (20% sensor noise)
Color: Number of Robots or Number of liars

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(1,10)

color_count=0
color_map=cm.Set1(np.linspace(0,1,51))

for key,value in density_converged_20.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('{} liars'.format(num_liars))
        boxplot_x=key+(0.5*counter)-0.5
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[num_liars]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.5)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[1.0,2.0,5.0,10.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Density')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 


**---**

# Converged Experiments with 40% sensor noise

### Dictionary Creation for successful seeds with 40 % sensor noise

In [ ]:
# Creating the needed dictionary
converged_40=Customdictionary()

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    if (noise==0.0):
        converged_40[successful_seeds[ind]]=pickle_dictionary[key_0][key_1]

### Dictionaries for timestep and density ratios

In [ ]:
time_converged_40=collections.defaultdict(dict)
density_converged_40=collections.defaultdict(dict)

for k,v in converged_40.items():
    k_id=k.split("_")
    num_robots=int(remove_chars(k_id[0]))
    num_liars=int(remove_chars(k_id[1]))
    num_working=num_robots-num_liars
    noise=float(remove_chars(k_id[2]))
    density=int(remove_chars(k_id[3]))
    seed=int(remove_chars(k_id[4]))
    
    key="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key2="_".join((key,"seed{}".format(seed)))
    raw_data=np.array(v).reshape((num_working,9))
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
    
    
    x_val=round(float(num_working/num_robots),2) # Working robots
    y_val=timestep
    #1
    if not keys_exists(time_converged_40,x_val,key):
        data=np.array(y_val)
        time_converged_40[x_val][key]=data
    else:
        data=time_converged_40[x_val][key]
        data=np.hstack((data,y_val))
        time_converged_40[x_val][key]=data
    #2
    if not keys_exists(density_converged_40,density,key):
        data=np.array(y_val)
        density_converged_40[density][key]=data
    else:
        data=density_converged_40[density][key]
        data=np.hstack((data,y_val))
        density_converged_40[density][key]=data


---

## Boxplot Time to converge vs. Working robot ratio 
Converged Experiments (40% sensor noise)
Color: Density or Number of Robots

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(0.4,1)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in time_converged_40.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('Density of {}'.format(density))
        boxplot_x=key+(0.01*counter)-0.015
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[density]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.005)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[0.2,0.25,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.88,0.9,0.95,1.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Working Robot Ratio')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 

---

## Boxplot Time to converge vs. Density
Converged Experiments (40% sensor noise)
Color: Number of Robots or Number of liars

In [ ]:
boxes=[]
keys=[]
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xlim(1,10)

color_count=0
color_map=cm.Set1(np.linspace(0,1,31))

for key,value in density_converged_40.items():
    counter=0
    for k2,v2 in value.items():
        identifier=k2.split("_")
        num_robots=int(remove_chars(identifier[0]))
        num_liars=int(remove_chars(identifier[1]))
        noise=float(remove_chars(identifier[2]))
        density=int(remove_chars(identifier[3]))
        
        ###
        #Colormap according to either TOTAL ROBOTS or DENSITY
        keys.append('{} total robots'.format(num_robots))
#         keys.append('{} liars'.format(num_liars))
        boxplot_x=key+(0.5*counter)-0.5
        counter+=1
        boxplot_y=np.array(v2).reshape((v2.size,1))
        color=color_map[num_robots]
#         color=color_map[num_liars]
        ###
        
        box=plt.boxplot(boxplot_y,patch_artist=True,positions=[boxplot_x],labels=[str(k2)],widths=0.5)
        boxes+=[box['boxes'][0]]
        for patch in (box['boxes']):
            patch.set_facecolor(color)
        
ticks=[1.0,2.0,5.0,10.0]
plt.xticks(ticks,ticks)        
plt.grid(True)
plt.xlabel('Density')
plt.ylabel('Timestep')
plt.legend(boxes,keys,loc='upper center',bbox_to_anchor=(0.5,-0.05),ncol=4)
plt.show() 

---
# Heatmap Successful Experiments

### Dictionary Creation for all successful seeds separated by sensor noise

In [ ]:
## Separating data based on sensor noise
heatmap_converged_dict_00=Customdictionary()
heatmap_converged_dict_05=Customdictionary()
heatmap_converged_dict_10=Customdictionary()
heatmap_converged_dict_20=Customdictionary()
heatmap_converged_dict_25=Customdictionary()
heatmap_converged_dict_30=Customdictionary()
heatmap_converged_dict_40=Customdictionary()

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)

    
    raw_data=pickle_dictionary[key_0][key_1]
    raw_data=np.array(raw_data).reshape((num_working,9)) #From pickle dictionary
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
  
    if (noise==0.0):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_00[key]=value
    elif (noise==0.05):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_05[key]=value
    elif (noise==0.10):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_10[key]=value
    elif (noise==0.20):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_20[key]=value
    elif (noise==0.25):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_25[key]=value
    elif (noise==0.30):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_30[key]=value
    elif (noise==0.40):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_converged_dict_40[key]=value
    else:
        print("ERROR")

In [ ]:
# Summaries
count_tstep_summary_dict_00=Customdictionary()
count_tstep_summary_dict_05=Customdictionary()
count_tstep_summary_dict_10=Customdictionary()
count_tstep_summary_dict_20=Customdictionary()
count_tstep_summary_dict_25=Customdictionary()
count_tstep_summary_dict_30=Customdictionary()
count_tstep_summary_dict_40=Customdictionary()

mean_tstep_summary_dict_00=Customdictionary()
mean_tstep_summary_dict_05=Customdictionary()
mean_tstep_summary_dict_10=Customdictionary()
mean_tstep_summary_dict_20=Customdictionary()
mean_tstep_summary_dict_25=Customdictionary()
mean_tstep_summary_dict_30=Customdictionary()
mean_tstep_summary_dict_40=Customdictionary()

for key, value in heatmap_converged_dict_00.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_00[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_00[key]=successful_experiments
    mean_tstep_summary_dict_00[key]=round(mean_tstep)
    
for key, value in heatmap_converged_dict_05.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_05[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_05[key]=successful_experiments
    mean_tstep_summary_dict_05[key]=round(mean_tstep)

for key, value in heatmap_converged_dict_10.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_10[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_10[key]=successful_experiments
    mean_tstep_summary_dict_10[key]=round(mean_tstep)

for key, value in heatmap_converged_dict_20.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_20[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_20[key]=successful_experiments
    mean_tstep_summary_dict_20[key]=round(mean_tstep)

for key, value in heatmap_converged_dict_25.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_25[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_25[key]=successful_experiments
    mean_tstep_summary_dict_25[key]=round(mean_tstep)

for key, value in heatmap_converged_dict_30.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_30[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_30[key]=successful_experiments
    mean_tstep_summary_dict_30[key]=round(mean_tstep)

for key, value in heatmap_converged_dict_40.items():
    key=key.split("_")[1]
    successful_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_40[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_40[key]=successful_experiments
    mean_tstep_summary_dict_40[key]=round(mean_tstep)

#### Pandas Count of successful experiments

In [ ]:
# Need to create an array from the summary dictionaries.
index=['0.00']
ct_df1=pd.DataFrame(data=count_tstep_summary_dict_00,index=index)
index=['0.05']
ct_df2=pd.DataFrame(data=count_tstep_summary_dict_05,index=index)
index=['0.10']
ct_df3=pd.DataFrame(data=count_tstep_summary_dict_10,index=index)
index=['0.20']
ct_df4=pd.DataFrame(data=count_tstep_summary_dict_20,index=index)
index=['0.25']
ct_df5=pd.DataFrame(data=count_tstep_summary_dict_25,index=index)
index=['0.30']
ct_df6=pd.DataFrame(data=count_tstep_summary_dict_30,index=index)
index=['0.40']
ct_df7=pd.DataFrame(data=count_tstep_summary_dict_40,index=index)

ct_df_all=ct_df7.append(ct_df6).append(ct_df5).append(ct_df4).append(ct_df3).append(ct_df2).append(ct_df1)

In [ ]:
ct_df_all

#### Pandas Mean timestep for successful experiments

In [ ]:
# Need to create an array from the summary dictionaries. Shoddy code
index=['0.00']
mt_df1=pd.DataFrame(data=mean_tstep_summary_dict_00,index=index)
index=['0.05']
mt_df2=pd.DataFrame(data=mean_tstep_summary_dict_05,index=index)
index=['0.10']
mt_df3=pd.DataFrame(data=mean_tstep_summary_dict_10,index=index)
index=['0.20']
mt_df4=pd.DataFrame(data=mean_tstep_summary_dict_20,index=index)
index=['0.25']
mt_df5=pd.DataFrame(data=mean_tstep_summary_dict_25,index=index)
index=['0.30']
mt_df6=pd.DataFrame(data=mean_tstep_summary_dict_30,index=index)
index=['0.40']
mt_df7=pd.DataFrame(data=mean_tstep_summary_dict_40,index=index)

mt_df_all=mt_df7.append(mt_df6).append(mt_df5).append(mt_df4).append(mt_df3).append(mt_df2).append(mt_df1)

## Heatmap Number of Successful Experiments [Sensor Noise vs. Defecting Ratio]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(ct_df_all, annot=True,cmap='hot')
plt.ylabel('Sensor Noise')
plt.xlabel('Defecting Ratio')
plt.title('Number of successful experiments')
# plt.savefig("SuccessfulExperiments_Random",bbox_inches='tight')
# plt.savefig("SuccessfulExperiments_Defecting",bbox_inches='tight')

## Heatmap Timestep of Successful Experiments [Sensor Noise vs. Defecting Ratio]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(mt_df_all, annot=True,cmap='summer')
plt.ylabel('Sensor Noise')
plt.xlabel('Defecting Ratio')
plt.title('Mean Timestep for completion')
# plt.savefig("Successful_MeanTimesteps_Random",bbox_inches='tight')
# plt.savefig("Successful_MeanTimesteps_Defecting",bbox_inches='tight')

---
# Heatmap Failed Experiments

### Dictionary Creation for all failed seeds separated by sensor noise

In [ ]:
## Separating data based on sensor noise
heatmap_nonconverged_dict_00=Customdictionary()
heatmap_nonconverged_dict_05=Customdictionary()
heatmap_nonconverged_dict_10=Customdictionary()
heatmap_nonconverged_dict_20=Customdictionary()
heatmap_nonconverged_dict_25=Customdictionary()
heatmap_nonconverged_dict_30=Customdictionary()
heatmap_nonconverged_dict_40=Customdictionary()

for ind in range(len(failed_seeds)):
    temp=failed_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)

    
    raw_data=pickle_dictionary[key_0][key_1]
   
    raw_data=np.array(raw_data).reshape((len(raw_data),9)) #From pickle dictionary
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)
  
    if (noise==0.0):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_00[key]=value
    elif (noise==0.05):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_05[key]=value
    elif (noise==0.10):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_10[key]=value
    elif (noise==0.20):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_20[key]=value
    elif (noise==0.25):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_25[key]=value
    elif (noise==0.30):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_30[key]=value
    elif (noise==0.40):
        str_noise=str(noise)
        str_defecting_robotratio=str(float(num_liars/num_robots))
        key="_".join((str_noise,str_defecting_robotratio))
        value=int(timestep)
        heatmap_nonconverged_dict_40[key]=value
    else:
        print("ERROR")

In [ ]:
# Summaries
count_tstep_summary_dict_00=Customdictionary()
count_tstep_summary_dict_05=Customdictionary()
count_tstep_summary_dict_10=Customdictionary()
count_tstep_summary_dict_20=Customdictionary()
count_tstep_summary_dict_25=Customdictionary()
count_tstep_summary_dict_30=Customdictionary()
count_tstep_summary_dict_40=Customdictionary()

mean_tstep_summary_dict_00=Customdictionary()
mean_tstep_summary_dict_05=Customdictionary()
mean_tstep_summary_dict_10=Customdictionary()
mean_tstep_summary_dict_20=Customdictionary()
mean_tstep_summary_dict_25=Customdictionary()
mean_tstep_summary_dict_30=Customdictionary()
mean_tstep_summary_dict_40=Customdictionary()

for key, value in heatmap_nonconverged_dict_00.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_00[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_00[key]=failed_experiments
    mean_tstep_summary_dict_00[key]=round(mean_tstep)
    
for key, value in heatmap_nonconverged_dict_05.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_05[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_05[key]=failed_experiments
    mean_tstep_summary_dict_05[key]=round(mean_tstep)

for key, value in heatmap_nonconverged_dict_10.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_10[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_10[key]=failed_experiments
    mean_tstep_summary_dict_10[key]=round(mean_tstep)

for key, value in heatmap_nonconverged_dict_20.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_20[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_20[key]=failed_experiments
    mean_tstep_summary_dict_20[key]=round(mean_tstep)

for key, value in heatmap_nonconverged_dict_25.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_25[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_25[key]=failed_experiments
    mean_tstep_summary_dict_25[key]=round(mean_tstep)

for key, value in heatmap_nonconverged_dict_30.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_30[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_30[key]=failed_experiments
    mean_tstep_summary_dict_30[key]=round(mean_tstep)

for key, value in heatmap_nonconverged_dict_40.items():
    failed_experiments=len(value)
    mean_tstep=np.mean(value)
#     summary_dict_40[key]=successful_experiments,round(mean_tstep)
    count_tstep_summary_dict_40[key]=failed_experiments
    mean_tstep_summary_dict_40[key]=round(mean_tstep)

#### Pandas Count of failed experiments

In [ ]:
# Need to create an array from the summary dictionaries.
index=['0.00']
ct_df1=pd.DataFrame(data=count_tstep_summary_dict_00,index=index)
index=['0.05']
ct_df2=pd.DataFrame(data=count_tstep_summary_dict_05,index=index)
index=['0.10']
ct_df3=pd.DataFrame(data=count_tstep_summary_dict_10,index=index)
index=['0.20']
ct_df4=pd.DataFrame(data=count_tstep_summary_dict_20,index=index)
index=['0.25']
ct_df5=pd.DataFrame(data=count_tstep_summary_dict_25,index=index)
index=['0.30']
ct_df6=pd.DataFrame(data=count_tstep_summary_dict_30,index=index)
index=['0.40']
ct_df7=pd.DataFrame(data=count_tstep_summary_dict_40,index=index)

ct_df_all=ct_df7.append(ct_df6).append(ct_df5).append(ct_df4).append(ct_df3).append(ct_df2).append(ct_df1)

In [ ]:
ct_df_all

#### Pandas Mean timestep for failed experiments

In [ ]:
# Need to create an array from the summary dictionaries. Shoddy code
index=['0.00']
mt_df1=pd.DataFrame(data=mean_tstep_summary_dict_00,index=index)
index=['0.05']
mt_df2=pd.DataFrame(data=mean_tstep_summary_dict_05,index=index)
index=['0.10']
mt_df3=pd.DataFrame(data=mean_tstep_summary_dict_10,index=index)
index=['0.20']
mt_df4=pd.DataFrame(data=mean_tstep_summary_dict_20,index=index)
index=['0.25']
mt_df5=pd.DataFrame(data=mean_tstep_summary_dict_25,index=index)
index=['0.30']
mt_df6=pd.DataFrame(data=mean_tstep_summary_dict_30,index=index)
index=['0.40']
mt_df7=pd.DataFrame(data=mean_tstep_summary_dict_40,index=index)

mt_df_all=mt_df7.append(mt_df6).append(mt_df5).append(mt_df4).append(mt_df3).append(mt_df2).append(mt_df1)

In [ ]:
mt_df_all

### Dictionary Creation for counts of failed robots

In [ ]:
count_failedrobot_ratio_dict_00=Customdictionary()
count_failedrobot_ratio_dict_05=Customdictionary()
count_failedrobot_ratio_dict_10=Customdictionary()
count_failedrobot_ratio_dict_20=Customdictionary()
count_failedrobot_ratio_dict_25=Customdictionary()
count_failedrobot_ratio_dict_30=Customdictionary()
count_failedrobot_ratio_dict_40=Customdictionary()

count_failedrobot_num_dict_00=Customdictionary()
count_failedrobot_num_dict_05=Customdictionary()
count_failedrobot_num_dict_10=Customdictionary()
count_failedrobot_num_dict_20=Customdictionary()
count_failedrobot_num_dict_25=Customdictionary()
count_failedrobot_num_dict_30=Customdictionary()
count_failedrobot_num_dict_40=Customdictionary()


for key,value in countfailed_robots_failedexp.items(): # Value is the number of failed robots
    fname=key.split("_")
    num_robots=int(remove_chars(fname[0]))
    num_liars=int(remove_chars(fname[1]))
    noise=float(remove_chars(fname[2]))
    density=float(remove_chars(fname[3]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    if (noise==0.0):
        count_failedrobot_ratio_dict_00[ratio_working]=value
        count_failedrobot_num_dict_00[num_working]=value
    elif (noise==0.05):
        count_failedrobot_ratio_dict_05[ratio_working]=value
        count_failedrobot_num_dict_05[num_working]=value
    elif (noise==0.10):
        count_failedrobot_ratio_dict_10[ratio_working]=value
        count_failedrobot_num_dict_10[num_working]=value
    elif (noise==0.20):
        count_failedrobot_ratio_dict_20[ratio_working]=value
        count_failedrobot_num_dict_20[num_working]=value
    elif (noise==0.25):
        count_failedrobot_ratio_dict_25[ratio_working]=value
        count_failedrobot_num_dict_25[num_working]=value
    elif (noise==0.30):
        count_failedrobot_ratio_dict_30[ratio_working]=value
        count_failedrobot_num_dict_30[num_working]=value
    elif (noise==0.40):
        count_failedrobot_ratio_dict_40[ratio_working]=value
        count_failedrobot_num_dict_40[num_working]=value

#### Pandas Count of failed robots vs. ratio of robots

In [ ]:
# Need to create an array from the summary dictionaries.
index=['0.00']
ct_f1_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_00,index=index)
index=['0.05']
ct_f2_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_05,index=index)
index=['0.10']
ct_f3_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_10,index=index)
index=['0.20']
ct_f4_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_20,index=index)
index_ratio=['0.25']
ct_f5_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_25,index=index)
index=['0.30']
ct_f6_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_30,index=index)
index=['0.40']
ct_f7_ratio=pd.DataFrame(data=count_failedrobot_ratio_dict_40,index=index)

ct_f_ratio_all=ct_f7_ratio.append(ct_f6_ratio).append(ct_f5_ratio).append(ct_f4_ratio).append(ct_f3_ratio).append(ct_f2_ratio).append(ct_f1_ratio)

In [ ]:
ct_f_ratio_all

#### Pandas Count of failed robots vs. number of working robots

In [ ]:
# Need to create an array from the summary dictionaries. Shoddy code
index=['0.00']
ct_f1_num=pd.DataFrame(data=count_failedrobot_num_dict_00,index=index)
index=['0.05']
ct_f2_num=pd.DataFrame(data=count_failedrobot_num_dict_05,index=index)
index=['0.10']
ct_f3_num=pd.DataFrame(data=count_failedrobot_num_dict_10,index=index)
index=['0.20']
ct_f4_num=pd.DataFrame(data=count_failedrobot_num_dict_20,index=index)
index=['0.25']
ct_f5_num=pd.DataFrame(data=count_failedrobot_num_dict_25,index=index)
index=['0.30']
ct_f6_num=pd.DataFrame(data=count_failedrobot_num_dict_30,index=index)
index=['0.40']
ct_f7_num=pd.DataFrame(data=count_failedrobot_num_dict_40,index=index)

ct_f_num_all=ct_f7_num.append(ct_f6_num).append(ct_f5_num).append(ct_f4_num).append(ct_f3_num).append(ct_f2_num).append(ct_f1_num)

In [ ]:
ct_f_num_all

## Heatmap Number of Failed Experiments [Sensor Noise vs. Defecting Ratio]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(ct_df_all, annot=True,cmap='hot')
plt.ylabel('Sensor Noise')
plt.xlabel('Defecting Ratio')
plt.title('Number of failed experiments')
# plt.savefig("SuccessfulExperiments_Random",bbox_inches='tight')
# plt.savefig("SuccessfulExperiments_Defecting",bbox_inches='tight')

## Heatmap Timestep of Failed Experiments [Sensor Noise vs. Defecting Ratio]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(mt_df_all, annot=True,cmap='summer')
plt.ylabel('Sensor Noise')
plt.xlabel('Defecting Ratio')
plt.title('Mean Timestep for completion- Best Time')
# plt.savefig("Successful_MeanTimesteps_Random",bbox_inches='tight')
# plt.savefig("Successful_MeanTimesteps_Defecting",bbox_inches='tight')

## Heatmap Number of Failed Robots [Sensor Noise vs. Defecting Ratio]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(ct_f_ratio_all, annot=True,cmap='summer')
plt.ylabel('Sensor Noise')
plt.xlabel('Defecting Ratio')
plt.title('Number of Failed Robots')
# plt.savefig("Successful_MeanTimesteps_Random",bbox_inches='tight')
# plt.savefig("Successful_MeanTimesteps_Defecting",bbox_inches='tight')

## Heatmap Number of Failed Robots [Sensor Noise vs. Working Robots]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(ct_f_num_all, annot=True,cmap='summer')
plt.ylabel('Sensor Noise')
plt.xlabel('Number of Working Robots')
plt.title('MNumber of Failed Robots')
# plt.savefig("Successful_MeanTimesteps_Random",bbox_inches='tight')
# plt.savefig("Successful_MeanTimesteps_Defecting",bbox_inches='tight')

---
# Scatter Plot of Messages Listened/Ignored

## Successful experiments

### Dictionaries for message counts and final beliefs

In [ ]:
scatter_messages_successfulexp={}
scatter_beliefs_successfulexp={}

for ind in range(len(successful_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    raw_data=pickle_dictionary[key_0][key_1]
    raw_data=np.array(raw_data).reshape((num_working,9)) #From pickle dictionary
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)

    key="_".join((key_0,key_1))
    for ind2 in range(num_working):
        key_robot="_".join((key,str(ind2))) # Creating a new key for every robot
        messagesdata_robot=[total_received_messages[ind2],total_pruned_messages[ind2],
                            total_lying_messages[ind2],total_lying_messages_believed[ind2]]
        probthresholddata_robot=[correct_pattern_prob[ind2],current_threshold[ind2]]
        
        scatter_messages_successfulexp[key_robot]=messagesdata_robot
        scatter_beliefs_successfulexp[key_robot]=probthresholddata_robot
    

### Scatter plot messages vs. num. robots

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_messages_successfulexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    recv_msg,pruned_msg,lying_msg,lying_msg_believed=val
    
    plt.scatter(num_working,recv_msg,c='blue',marker='o') # Total received messages blue circles
    plt.scatter(num_working,pruned_msg,c='green',marker='o') # Total pruned messages green circles
    plt.scatter(num_working,lying_msg,c='orange',marker='*') # Lying messages orange asterisks
    plt.scatter(num_working,lying_msg_believed,c='red',marker='*') # Lying messages believed red asterisks
    

plt.ylabel('Message Count')
plt.xlabel('Number of working robots')
plt.title('Scatter plot of messages vs. robots successful experiments')
plt.grid(True)
plt.show()

### Scatter plot messages vs. working robot ratio

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_messages_successfulexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    recv_msg,pruned_msg,lying_msg,lying_msg_believed=val
    
    plt.scatter(ratio_working,recv_msg,c='blue',marker='o') # Total received messages blue circles
    plt.scatter(ratio_working,pruned_msg,c='green',marker='o') # Total pruned messages green circles
    plt.scatter(ratio_working,lying_msg,c='orange',marker='*') # Lying messages orange asterisks
    plt.scatter(ratio_working,lying_msg_believed,c='red',marker='*') # Lying messages believed red asterisks
    

plt.ylabel('Message Count')
plt.xlabel('Ratio of working robots')
plt.title('Scatter plot of messages vs. ratio of working robots successful experiments')
plt.grid(True)
plt.show()

### Scatter plot beliefs vs. num. robots

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_beliefs_successfulexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    correct_pattern_prob,current_threshold=val
    
    plt.scatter(num_working,correct_pattern_prob,c='green',marker='o') # Final belief probability green circle
    plt.scatter(num_working,current_threshold,c='blue',marker='*') # Robot threshold blue asterisk
    
plt.ylabel('Probability')
plt.xlabel('Number of working robots')
plt.title('Scatter plot of probability vs. robots successful experiments')
plt.grid(True)
plt.show()

### Scatter plot beliefs vs. working robot ratio

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_beliefs_successfulexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    correct_pattern_prob,current_threshold=val
    
    plt.scatter(ratio_working,correct_pattern_prob,c='green',marker='o') # Final belief probability green circle
    plt.scatter(ratio_working,current_threshold,c='blue',marker='*') # Robot threshold blue asterisk
    
plt.ylabel('Probability')
plt.xlabel('Ratio of working robots')
plt.title('Scatter plot of probability vs. ratio of working robots successful experiments')
plt.grid(True)
plt.show()

## Failed experiments

### Dictionaries for message counts and final beliefs

In [ ]:
scatter_messages_failedexp={}
scatter_beliefs_failedexp={}

for ind in range(len(failed_seeds)):
    temp=successful_seeds[ind].split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    key_0="nrobots{}_nliars{}_noise{}_density{}".format(num_robots,num_liars,noise,density)
    key_1="seed{}".format(seed)
    
    raw_data=pickle_dictionary[key_0][key_1]
    raw_data=np.array(raw_data).reshape((num_working,9)) #From pickle dictionary
        
    timestep=raw_data[0,0]
    correct_pattern_prob=raw_data[:,2]
    total_received_messages=raw_data[:,4]
    total_pruned_messages=raw_data[:,5]
    total_lying_messages=raw_data[:,6]
    total_lying_messages_believed=raw_data[:,7]
    current_threshold=raw_data[:,8] 

    average_pattern_prob=np.mean(correct_pattern_prob)
    average_sum_received=np.mean(total_received_messages)
    average_sum_pruned=np.mean(total_pruned_messages)
    average_sum_lying=np.mean(total_lying_messages)
    average_sum_lying_believed=np.mean(total_lying_messages_believed)
    average_threshold=np.mean(current_threshold)

    key="_".join((key_0,key_1))
    for ind2 in range(num_working):
        key_robot="_".join((key,str(ind2))) # Creating a new key for every robot
        messagesdata_robot=[total_received_messages[ind2],total_pruned_messages[ind2],
                            total_lying_messages[ind2],total_lying_messages_believed[ind2]]
        probthresholddata_robot=[correct_pattern_prob[ind2],current_threshold[ind2]]
        
        scatter_messages_failedexp[key_robot]=messagesdata_robot
        scatter_beliefs_failedexp[key_robot]=probthresholddata_robot
    

### Scatter plot messages vs. num. robots

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_messages_failedexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    recv_msg,pruned_msg,lying_msg,lying_msg_believed=val
    
    plt.scatter(num_working,recv_msg,c='blue',marker='o') # Total received messages blue circles
    plt.scatter(num_working,pruned_msg,c='green',marker='o') # Total pruned messages green circles
    plt.scatter(num_working,lying_msg,c='orange',marker='*') # Lying messages orange asterisks
    plt.scatter(num_working,lying_msg_believed,c='red',marker='*') # Lying messages believed red asterisks
    

plt.ylabel('Message Count')
plt.xlabel('Number of working robots')
plt.title('Scatter plot of messages vs. robots failed experiments')
plt.grid(True)
plt.show()

### Scatter plot messages vs. working robot ratio

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_messages_failedexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    recv_msg,pruned_msg,lying_msg,lying_msg_believed=val
    
    plt.scatter(ratio_working,recv_msg,c='blue',marker='o') # Total received messages blue circles
    plt.scatter(ratio_working,pruned_msg,c='green',marker='o') # Total pruned messages green circles
    plt.scatter(ratio_working,lying_msg,c='orange',marker='*') # Lying messages orange asterisks
    plt.scatter(ratio_working,lying_msg_believed,c='red',marker='*') # Lying messages believed red asterisks
    

plt.ylabel('Message Count')
plt.xlabel('Ratio of working robots')
plt.title('Scatter plot of messages vs. ratio of working robots failed experiments')
plt.grid(True)
plt.show()

### Scatter plot beliefs vs. num. robots

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_beliefs_failedexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    correct_pattern_prob,current_threshold=val
    
    plt.scatter(num_working,correct_pattern_prob,c='green',marker='o') # Final belief probability green circle
    plt.scatter(num_working,current_threshold,c='blue',marker='*') # Robot threshold blue asterisk
    
plt.ylabel('Probability')
plt.xlabel('Number of working robots')
plt.title('Scatter plot of probability vs. robots failed experiments')
plt.grid(True)
plt.show()

### Scatter plot beliefs vs. working robot ratio

In [ ]:
plt.figure(figsize=(20,15),dpi=100)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
# plt.xlim(1,10)

for key,val in scatter_beliefs_failedexp.items():
    temp=key.split("_")
    num_robots=int(remove_chars(temp[0]))
    num_liars=int(remove_chars(temp[1]))
    noise=float(remove_chars(temp[2]))
    density=int(remove_chars(temp[3]))
    seed=int(remove_chars(temp[4]))
    num_working=num_robots-num_liars
    ratio_working=float(num_working/num_robots)
    
    # Total
    correct_pattern_prob,current_threshold=val
    
    plt.scatter(ratio_working,correct_pattern_prob,c='green',marker='o') # Final belief probability green circle
    plt.scatter(ratio_working,current_threshold,c='blue',marker='*') # Robot threshold blue asterisk
    
plt.ylabel('Probability')
plt.xlabel('Ratio of working robots')
plt.title('Scatter plot of probability vs. ratio of working robots failed experiments')
plt.grid(True)
plt.show()

---
# BREAK